In [1]:
import cluj_tools as ct

from datetime import timedelta, datetime

### Pull yesterdays boxscores

In [2]:
day = datetime.now() - timedelta(days=1)
print(day)

2020-02-11 07:00:04.603048


In [3]:
ct.pull_boxscores(day)

(111, 17)
